In [1]:
from llama_cloud_services import LlamaCloudIndex
from llama_index.llms.ollama import Ollama
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import asyncio
import os
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools import FunctionTool

/media/aumoza/Strg_1/Freelance/personal/Chatbot/chatbot/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/media/aumoza/Strg_1/Freelance/personal/Chatbot/chatbot/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
model = Ollama(
    model="llama3.2:latest",
    request_timeout=120.0,
    context_window=8000
)

In [6]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

/media/aumoza/Strg_1/Freelance/personal/Chatbot/chatbot/lib/python3.13/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [7]:
documents = SimpleDirectoryReader("data").load_data()

In [8]:
index = VectorStoreIndex.from_documents(
    documents,
    # we can optionally override the embed_model here
    # embed_model=Settings.embed_model,
)

In [13]:
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    llm = model,
    chat_mode="context",
    memory=memory,
    system_prompt = (
    "You are a helpful and friendly shopping assistant chatbot. "
),
)

In [19]:
response = chat_engine.chat("Yeah, now I want to purchase a dining table, what options do you have?")

In [20]:
response.response

"I'm glad we can move on to finding the perfect dining table for you.\n\nFrom our previous data, we have one dining table available:\n\n1. **Modern Wooden Dining Table**:\n   - Category: Dining\n   - Price: ₹ 18,999\n   - Currency: INR\n   - Warranty: 2 years\n   - Manufacturer: UrbanCraft Furnitures\n   - Material: Sheesham Wood\n   - Color: Walnut Brown\n   - Dimensions: Length (cm) = 180, Width (cm) = 90, Height (cm) = 75\n   - Weight: 35 kg\n   - Availability: In Stock\n   - Delivery Time: 5-7 business days\n   - Return Policy: 10-day replacement\n\nAdditionally, I'd like to let you know that this dining table is a popular option among our customers. However, if you're looking for other options or different specifications, please let me know and I can check if there are any other products available.\n\nWould you like to learn more about this product or would you like me to explore other options?"